In [16]:
import requests
from bs4 import BeautifulSoup
import re , json
import time
from tqdm import tqdm
import pandas as pd
from manage_csv_1 import reset_csv_dataset, searchValue

lst_state=['selangor']#, 'kuala-lumpur']

lst_place_type=['all-residential'] #, 'all-commercial', 'all-industrial', 'all-agricultural']
lst_type=['sale']
attr = ["prices", "address", "attributes", "multilanguagePlace"]

max_page = 1
ds_name='data_new.csv'

headers = """
authority: www.iproperty.com.my
method: GET
path: /
scheme: https
accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: gzip, deflate, br
accept-language: en-US,en;q=0.9,fa;q=0.8,de;q=0.7
cache-control: max-age=0
cookie: feature-RWD-6307=true; feature-RWD-6862-1=control; feature-RWD-7170-1=true; feature-RWD-7145-1=true; feature-RWD-7252=control; feature-RWCP-24-1=true; feature-RWD-7274-1=true; feature-RWD-7197=control; feature-RWD-7571=control; _gcl_au=1.1.868914632.1614006197; _ga=GA1.3.670177510.1614006197; _fbp=fb.2.1614006198232.1085373626; _hjTLDTest=1; _hjid=05f07f56-4413-4583-bd03-a0cd6d13505e; __gads=ID=e012a425ddde61d5:T=1614006202:S=ALNI_MY1MNZ3hVX_uRrGNgsJigg9KnLgIw; scarab.visitor=%2228985172CA480CF7%22; ASP.NET_SessionId=hr1o2fly2veh505egdv5t5rx; anonymoususerid=5850c23b-600b-4ebc-9522-7fbf23959a0b; scarab.profile=%225903%7C1614012352%22; scarab.mayAdd=%5B%7B%22i%22%3A%225903%22%7D%2C%7B%22i%22%3A%22101584625%22%7D%5D; KP_UID=ec1361ad-fadb-ee62-3314-88f020b1dcd2; _gid=GA1.3.1896028716.1614154337; _vwo_uuid_v2=DFC6A03DF5B1902DD79575D3D53C0678B|705d37525459f9f466b5d25e700e9aa8; __utmc=108855130; adsblock=yes; _knxcm_=googleT1614165346.0004T1614165346.0005T1614165346.0006T1614165346.0008T1614165346.0010T1614165346.0011T1614165346.0013T1614165346.0015T1614165346.0016T1614165346.0011T1614165346; cloudfront_viewer_country=GB; _hjIncludedInPageviewSample=1; _hjIncludedInSessionSample=1; __utma=108855130.670177510.1614006197.1614165343.1614171002.2; __utmz=108855130.1614171002.2.2.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); gdpr_notification=yes; _dc_gtm_UA-85157164-4=1; _gat_UA-85157164-4=1; KP_UIDz=bCpm0h5msVzQ7kaqjtRMjQ%3D%3D%3A%3AJU1Q05CSfG6%2FRUhS3Ci8t%2BF6WcvATd7IKkYJGxhJ5dxmZnyRiYIv0CECGWCkMnTz77g2K%2FLcq%2FOFt80CMVM9lzkdXh7iCoRydtmNXg9B6ruLhFrSqXe4iq%2Bre81FeRTckMxpvHrTswP%2FRbKKE5JkWBcahjRdcxBC%2FS4ZaIFN2oZh%2FUhSUsNjXfS1erO1NzoXzne6thIvucSHg1nDJ%2F5TjtDItyb6mswXms3kpe4rI4wtEJNl42tr1S4SmFo2OAlGcX0GrelcZMaSDTK1VSl9cpPK66zvfKN0t6BLFACOUjfFTl7mrmq0YhsF2VH8Pd6uc0PRLhz%2BkvQENIOrEG1Zt1fWupx7kEzE1INpeXJAkzdlkuUNnFMnFQ2orC1gIwG0dfv1a1vuxW6ClPjBVl310RcsT6sC%2BVaZ1EMVwP7z%2F5vjCzJw%2FgNmyw%2Fk4Wc%2FNXsfdce%2FAd%2BarQ03GqmExhOuOjUcHMA4Dn3qt6sWm1uA2bEDxIaZNeARKwcY1EtuZQj1%2B%2B5O8NQnQJFL1GLbPN7ZcpYxUge60IPZKz60KgvkktiiYvQRus2j1cO1OnzV5qo3
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: none
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36
"""

combined = {}

for row in headers.split('\n')[1:-1]:
    splitted = row.split(':')
    key = splitted[0]
    val = ''.join(splitted[1:])
    combined[key] = val.strip()


dict_data={'state': [],
           'type' : [],
           'max_price' : [],
           'min_price' : [],
           'address' : [],
           'lat' : [],
           'lng' : [],
           'landArea' : [],
           'builtUp' : [],
           'sizeUnit' : [],
           'level1' : [],
           'level2' : [],
           'level3' : []}

reset_csv_dataset()

for i_type in range(len(lst_type)):
    _type = lst_type[i_type]
    for i_state in range(len(lst_state)):
        _state = lst_state[i_state]
        start_time_State = time.time()
        for i_ptype in range(len(lst_place_type)):
            _ptype = lst_place_type[i_ptype]
            for i_page in tqdm(range(1,max_page+1)):
                try:
                    start_time = time.time()
                    url = f'https://www.iproperty.com.my/{_type}/{_state}/{_ptype}/?l1&page={i_page}'
                    r = requests.get(url, headers=combined)
                    # print(f'\n{url}\nstatus_code: ', r.status_code, r.reason)

                    bs = BeautifulSoup(r.content, 'html5lib')

                    pattern = re.compile('window.__INITIAL_STATE__')
                    script = bs.find("script", text=pattern)

                    data = re.search(r'window\.__INITIAL_STATE__ = ({.*});', script.contents[0]).group(1)
                    print(data)
                    data = '{"prices"' + re.search(r'"items":\[\{"prices"(.*)}],"isFetching"', data).group(1) + '}]}'

                    data=data.split('{"prices"')
                    data.pop(0)
                    for i in range(len(data)):
                        data[i]='{"prices"'+data[i]

                        data[i] = data[i][:-1]
                        if len(data)== i+1:
                            data[i] = data[i][:-2] + '}'


                    for item in range(len(data)):
                        js_data = json.loads(data[item])
                        print(js_data['prices'])

                        i=attr[0]

                        res_type_price = searchValue(js_data[i], varName='Type', str_regex=r"'type': '(.*?)',")
                        res_min_price = searchValue(js_data[i], varName='min Price', str_regex=r"'min': (.*?),", bln_isNumber=True)
                        res_max_price = searchValue(js_data[i], varName='max Price', str_regex=r"'max':(.*?)}", bln_isNumber=True)

                        i=attr[1]
                        # print(f'{i}= "{str(js_data[i])}"')

                        res_address = searchValue(js_data[i], varName='Formatted Address', str_regex=r"'formattedAddress': '(.*?)',")
                        res_lat = searchValue(js_data[i], varName='Lat', str_regex=r"'lat': (.*?),", bln_isNumber=True)
                        res_lng = searchValue(js_data[i], varName='Lng', str_regex=r"'lng': (.*?),", bln_isNumber=True)

                        i=attr[2]
                        # print(f'{i}= "{str(js_data[i])}"')

                        res_landArea = searchValue(js_data[i], varName='Land Area', str_regex=r"'landArea': '(.*?)',", bln_isNumber=True)
                        res_builtUp = searchValue(js_data[i], varName='Built Up', str_regex=r"'builtUp': '(.*?)',", bln_isNumber=True)
                        res_sizeUnit = searchValue(js_data[i], varName='Size Unit', str_regex=r"'sizeUnit': '(.*?)',")

                        i=attr[3]
                        # print(f'{i}= "{str(js_data[i])}"')

                        res_level1 = searchValue(js_data[i], varName='level1', str_regex=r"'level1': '(.*?)'")
                        res_level2 = searchValue(js_data[i], varName='level2', str_regex=r"'level2': '(.*?)'")
                        res_level3 = searchValue(js_data[i], varName='level3', str_regex=r"'level3': '(.*?)'")

                        # print(f'{item}) type: {res_type_price}, min: {res_min_price}, max: {res_max_price}, formattedAddress: {res_address}, '
                        #       +f'lat: {res_lat}, lng: {res_lng}, landArea: {res_landArea}, builtUp: {res_builtUp}, sizeUnit: {res_sizeUnit}, '
                        #       +f'level1: {res_level1}, level2: {res_level2}, level3: {res_level3}')

                        dict_data['state'] = _state
                        dict_data['type'] = _type
                        dict_data['max_price'] = res_max_price
                        dict_data['min_price'] = res_min_price
                        dict_data['address'] = res_address
                        dict_data['lat'] = res_lat
                        dict_data['lng'] = res_lng
                        dict_data['landArea'] = res_landArea
                        dict_data['builtUp'] = res_builtUp
                        dict_data['sizeUnit'] = res_sizeUnit
                        dict_data['level1'] = res_level1
                        dict_data['level2'] = res_level2
                        dict_data['level3'] = res_level3


                        df = pd.read_csv(ds_name)
                        # print(df)
                        df = df.append(dict_data, ignore_index=True)
                        df.to_csv(ds_name, index_label=False)
                except Exception as e:
                    print(f'\n{url}\nError: Type: {_type}, State: {_state}, ptype: {_ptype}, Page:{i_page}, Error: {e}')
                print(f"{_state} {_type} --- Page  %s seconds ---" % (time.time() - start_time))
        print(f"\nState:{_state} {_type} --- %s seconds ---" % (time.time() - start_time_State))

  0%|                                                     | 0/1 [00:00<?, ?it/s]

{"asyncData":{},"agentProfile":{"listings":{"items":[],"filters":{"channel":"sale","pageSize":9,"pageNumber":1,"isIncludeListingSummary":true,"facetType":"agent-profile","locations":null,"propertyTypes":null,"minPrice":null,"maxPrice":null,"sortBy":null},"summary":{"sale":{"totalCount":0},"rent":{"totalCount":0}}}},"detail":{"renderUid":"MlG-BZA02X"},"organisation":{"renderUid":"MlG-BZA02X","popularPlaces":{"items":[{"slugId":"selangor","level1":"Selangor","level2":null,"level3":null},{"slugId":"johor","level1":"Johor","level2":null,"level3":null},{"slugId":"kuala-lumpur","level1":"Kuala Lumpur","level2":null,"level3":null},{"slugId":"negeri-sembilan","level1":"Negeri Sembilan","level2":null,"level3":null},{"slugId":"kedah","level1":"Kedah","level2":null,"level3":null},{"slugId":"pahang","level1":"Pahang","level2":null,"level3":null},{"slugId":"penang","level1":"Penang","level2":null,"level3":null},{"slugId":"kelantan","level1":"Kelantan","level2":null,"level3":null},{"slugId":"melaka"

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.74s/it]

[{'type': 'sale', 'currency': 'MYR', 'min': 790000, 'max': 790000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 550000, 'max': 550000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 830000, 'max': 830000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 1350000, 'max': 1350000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 298000, 'max': 298000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 325000, 'max': 325000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 1750000, 'max': 1750000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 950000, 'max': 950000}]
[{'type': 'sale', 'currency': 'MYR', 'min': 1650000, 'max': 1650000}]
selangor sale --- Page  3.735383987426758 seconds ---

State:selangor sale --- 3.757566213607788 seconds ---


In [17]:
df.head()

,state,type,max_price,min_price,address,lat,lng,landArea,builtUp,sizeUnit,level1,level2,level3
0,selangor,sale,780000,780000,"Kota Warisan, 43900, Selangor",NaN,NaN,3498,1848,SQUARE_FEET,Selangor,Sepang,NaN
1,selangor,sale,3280000,3280000,"Elitis, Valencia, 47000, Selangor",NaN,NaN,6404,4200,SQUARE_FEET,Selangor,Valencia,NaN
2,selangor,sale,1680000,1680000,"43, Jalan Damai Perdana 13/1E, Bandar Damai Pe...",3.045099,101.736303,3150,4200,SQUARE_FEET,Selangor,Cheras,NaN
3,selangor,sale,0,681044,"Klang, Selangor",3.083019,101.428827,20’x75’,From 1718 - 2803,NaN,Selangor,Klang,Bandar Bukit Raja : Lumira
4,selangor,sale,550000,550000,"Jalan PJU 1A/3, Ara Damansara, 47301, Selangor",3.122416,101.589154,NaN,810,SQUARE_FEET,Selangor,Ara Damansara,Maisson


In [18]:
df.shape

(24, 13)

In [19]:
df.isna().sum()

state         0
type          0
max_price     0
min_price     0
address       0
lat           8
lng           8
landArea      7
builtUp       0
sizeUnit      4
level1        0
level2        0
level3       14
dtype: int64

In [11]:
df.state.unique()

array(['selangor', 'kuala-lumpur'], dtype=object)